In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
import os
from hstrader import HsTrader
from hstrader.models import Tick, Event, Resolution
from dotenv import load_dotenv
import logging
import asyncio
from IPython.display import display, clear_output
import nest_asyncio

In [ ]:
# Enable logging
logging.basicConfig(level=logging.INFO)

# Load environment variables
load_dotenv()

# Get the CLIENT_ID and CLIENT_SECRET from the environment variables
id = os.getenv('CLIENT_ID')
secret = os.getenv('CLIENT_SECRET')

# Initialize the HsTrader client with the client ID and secret
client = HsTrader(id, secret)

In [ ]:
# Ichimoku Cloud calculations
def calculate_ichimoku(df):
    high_9 = df['high'].rolling(window=9).max()
    low_9 = df['low'].rolling(window=9).min()
    df['tenkan_sen'] = (high_9 + low_9) / 2

    high_26 = df['high'].rolling(window=26).max()
    low_26 = df['low'].rolling(window=26).min()
    df['kijun_sen'] = (high_26 + low_26) / 2

    df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)
    high_52 = df['high'].rolling(window=52).max()
    low_52 = df['low'].rolling(window=52).min()
    df['senkou_span_b'] = ((high_52 + low_52) / 2).shift(26)

    df['chikou_span'] = df['close'].shift(-26)
    return df

### Plot Candlestick Chart with Ichimoku Cloud Indicator (Historical Market Data):

In [ ]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol = client.get_symbol('ETHXBT')
data = client.get_market_history(symbol=symbol.id, resolution=Resolution.M1)

# Create a DataFrame from the retrieved data
df = pd.DataFrame([bar.model_dump() for bar in data])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

In [ ]:
# Create a candlestick chart using Plotly
candle=go.Candlestick(x=df.index,open=df['open'].shift(-26),high=df['high'].shift(-26),
                      low=df['low'].shift(-26),close=df['close'].shift(-26),name='Candlestick')

In [ ]:
# Define a function to determine the fill color for the Ichimoku Cloud
def get_fill_color(label):
    # Green color for bullish signal, red for bearish
    return 'rgba(0,255,0,0.4)' if label == 1 else 'rgba(255,0,0,0.4)'

# Initialize a new figure for plotting
fig = go.Figure()

df1 = calculate_ichimoku(df)

# Create a 'label' column to identify bullish (1) and bearish (0) areas
df1['label'] = np.where(df1['senkou_span_a'] > df1['senkou_span_b'], 1, 0)

# Group the data by consecutive labels to create continuous areas
df1['group'] = df1['label'].ne(df1['label'].shift()).cumsum()
groups = df1.groupby('group')

# Split the dataframe into subframes for each group
dfs = [data for _, data in groups]

# Plot each group's Ichimoku Cloud area
for df in dfs:
    # Add the Senkou Span A line (invisible, used for filling color)
    fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_a'], 
                             line=dict(color='rgba(0,0,0,0)'), name='cloud', showlegend=False))
    # Add the Senkou Span B line and fill the area between them
    fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_b'], 
                             line=dict(color='rgba(0,0,0,0)'), fill='tonexty', 
                             fillcolor=get_fill_color(df['label'].iloc[0]), name='cloud', showlegend=False))

# Plot the Kijun-sen line
Kijun_sen = go.Scatter(x=df1.index, y=df1['kijun_sen'].shift(-26), 
                       line=dict(color='red', width=1), name='kijun_sen')

# Plot the Tenkan-sen line
tenkan_sen = go.Scatter(x=df1.index, y=df1['tenkan_sen'].shift(-26), 
                        line=dict(color='blue', width=1), name='Tenkan_sen')

# Plot the Chikou Span line
chikou_span = go.Scatter(x=df1.index, y=df1['chikou_span'].shift(-26), 
                         line=dict(color='green', width=1), name='Chikou_span')

# Plot the Senkou Span A line
senkou_span_A = go.Scatter(x=df1.index, y=df1['senkou_span_a'], 
                           line=dict(color='green', width=1), name='Senkou span A')

# Plot the Senkou Span B line
senkou_span_B = go.Scatter(x=df1.index, y=df1['senkou_span_b'], 
                           line=dict(color='red', width=1), name='Senkou span B')

# Add the candlestick trace
fig.add_trace(candle)

# Add all the Ichimoku lines to the figure
fig.add_trace(Kijun_sen)
fig.add_trace(tenkan_sen)
fig.add_trace(chikou_span)
fig.add_trace(senkou_span_A)
fig.add_trace(senkou_span_B)

# Update the layout of the figure
fig.update_layout(width=1100, height=400, xaxis_rangeslider_visible=False, showlegend=True)

# Display the figure
fig.show()


### Example Plot

Below is a static image of the Ichimoku Cloud applied to historical data:

![Final Plot](img/ichimoku_live.png)

### Plot Real-Time Candlestick Chart with Ichimoku Cloud Indicator :

In [ ]:
# Retrieve market data for any forex symbol by specifying its name in the get_symbol function
symbol = client.get_symbol('ETHXBT')
data = client.get_market_history(symbol=symbol.id, resolution=Resolution.M1)

# Create a DataFrame from the retrieved data
df = pd.DataFrame([bar.model_dump() for bar in data])
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

In [ ]:
df = calculate_ichimoku(df)

# Create a Figure
fig = go.Figure()

candlestick = go.Candlestick(
    x=df.index, open=df['open'], high=df['high'].shift(-26), low=df['low'].shift(-26), close=df['close'].shift(-26), name='Candlestick'
)
fig.add_trace(candlestick)

# Add Ichimoku Cloud traces
fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_a'], mode='lines', name='Senkou Span A'))
fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_b'], mode='lines', name='Senkou Span B'))

# Function to add the filled area with dynamic colors
def add_dynamic_filled_areas(fig, df):
    prev_fill_color = None
    x_segment = []
    y_segment_a = []
    y_segment_b = []
    for i in range(len(df)):
        fill_color = 'rgba(0, 250, 0, 0.4)' if df['senkou_span_a'].iloc[i] > df['senkou_span_b'].iloc[i] else 'rgba(255, 0, 0, 0.4)'
        if fill_color != prev_fill_color and prev_fill_color is not None:
            # Add a trace for the previous segment
            fig.add_trace(go.Scatter(
                x=x_segment + x_segment[::-1],
                y=y_segment_a + y_segment_b[::-1],
                fill='toself',
                fillcolor=prev_fill_color,
                line=dict(color='rgba(255,255,255,0)'),
                showlegend=False
            ))
            x_segment = []
            y_segment_a = []
            y_segment_b = []
        x_segment.append(df.index[i])
        y_segment_a.append(df['senkou_span_a'].iloc[i])
        y_segment_b.append(df['senkou_span_b'].iloc[i])
        prev_fill_color = fill_color

    # Add the last segment
    if x_segment:
        fig.add_trace(go.Scatter(
            x=x_segment + x_segment[::-1],
            y=y_segment_a + y_segment_b[::-1],
            fill='toself',
            fillcolor=prev_fill_color,
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=False
        ))

add_dynamic_filled_areas(fig, df.shift(26))

# Prepare data dictionary
CANDLE_INTERVAL = timedelta(minutes=1)
data = {
    'x': list(df.index),
    'open': list(df['open']),
    'high': list(df['high']),
    'low': list(df['low']),
    'close': list(df['close'])
}

@client.subscribe(Event.MARKET)
async def on_market(tick: Tick):
    global data, df
    try:
        if tick.symbol_id == symbol.id:
            tick_time = pd.to_datetime(tick.time)
            if not data['x']:
                data['x'].append(tick_time)
                data['open'].append(data['close'][-1])
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
            elif tick_time >= data['x'][-1] + CANDLE_INTERVAL:
                data['x'].append(tick_time)
                data['open'].append(data['close'][-1])
                data['high'].append(tick.bid)
                data['low'].append(tick.bid)
                data['close'].append(tick.bid)
            else:
                data['low'][-1] = min(tick.bid, data['low'][-1])
                data['high'][-1] = max(tick.bid, data['high'][-1])
                data['close'][-1] = tick.bid

            # Update DataFrame with the latest tick data
            df = pd.DataFrame({
                'time': data['x'],
                'open': data['open'],
                'high': data['high'],
                'low': data['low'],
                'close': data['close']
            }).set_index('time')
            
            # Recalculate Ichimoku Cloud
            df = calculate_ichimoku(df)
    except Exception as e:
        logging.error(f"Error in on_market: {e}")
        await client.refresh_token()

async def update_plot():
    global fig
    
    layout = fig.layout
    while True:
        await asyncio.sleep(1)
        clear_output(wait=True)
        
        # Extend DataFrame to include future dates for plotting
        future_dates = pd.date_range(start=df.index[-1], periods=27, freq='min')[1:]
        future_df = pd.DataFrame(index=future_dates)
        df_extended = pd.concat([df, future_df])

        new_fig = go.Figure()

        # Shift the data by -26
        shifted_data = pd.DataFrame({
            'x': data['x'],
            'open': data['open'],
            'high': data['high'],
            'low': data['low'],
            'close': data['close']
        }).shift(-26)

        # Add updated candlestick trace
        new_fig.add_trace(go.Candlestick(
            x=shifted_data['x'], open=shifted_data['open'], high=shifted_data['high'], low=shifted_data['low'], close=shifted_data['close'], name='Candlestick'
        ))

        # Add updated Ichimoku Cloud traces
        new_fig.add_trace(go.Scatter(x=df_extended.index, y=df_extended['senkou_span_a'].shift(26), mode='lines', name='Senkou Span A'))
        new_fig.add_trace(go.Scatter(x=df_extended.index, y=df_extended['senkou_span_b'].shift(26), mode='lines', name='Senkou Span B'))

        # Add dynamic filled areas
        add_dynamic_filled_areas(new_fig, df_extended.shift(26))

        # Add the rest of the Ichimoku Cloud components
        new_fig.add_trace(go.Scatter(x=df.index, y=df['tenkan_sen'], mode='lines', name='Tenkan-sen'))
        new_fig.add_trace(go.Scatter(x=df.index, y=df['kijun_sen'], mode='lines', name='Kijun-sen'))
        new_fig.add_trace(go.Scatter(x=df.index, y=df['chikou_span'], mode='lines', name='Chikou Span'))

        # Apply previous layout to maintain zoom/pan state
        new_fig.layout = layout

        display(new_fig)
        layout = new_fig.layout
        
nest_asyncio.apply()
loop=asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(client.start_async(), update_plot()))